# Carregar dados

In [3]:
import pandas as pd

# # Mostrar mais colunas
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', 100)

df = pd.read_parquet("./output/silver/rais_TODOS_ANOS_RAIS_VINC_PUB.parquet.gzip")


qnt_full = len(df)
print(f"""
Quantidade docs: {qnt_full}
""")

df.head(2)



Quantidade docs: 1543009



,ano,id_uf_municipio,remuneracao_media,desligamento_id,idade,ocupacao_id,sexo_id,sigla_uf,sigla_municipio,valor_remuneracao_media,raca_cor_id
0,2019,500240,997.99,12.0,17,317210,1,MS,Caarapo,NaN,NaN
1,2019,500570,0.00,11.0,36,212420,1,MS,Navirai,NaN,NaN


In [4]:
df_sexo = pd.read_csv("./output/bronze/planilhas/dic_sexo.csv")
df_ordered_sinonimo = pd.read_csv("./output/bronze/planilhas/ordered_sinonimo.csv")

In [5]:
# change column ocupacao_id to int
df["ocupacao_id"] = df["ocupacao_id"].astype(int)

# Juntando tabelas

In [6]:

df = pd.merge(df, df_sexo, left_on="sexo_id", right_on="chave",
              how="left")
df = pd.merge(df, df_ordered_sinonimo, left_on="ocupacao_id",
              right_on="cbo_2002", how="left")
df.head(2)

,ano,id_uf_municipio,remuneracao_media,desligamento_id,idade,ocupacao_id,sexo_id,sigla_uf,sigla_municipio,valor_remuneracao_media,raca_cor_id,chave,valor,cbo_2002,sinonimo
0,2019,500240,997.99,12.0,17,317210,1,MS,Caarapo,NaN,NaN,1,Masculino,317210.0,Analista de service desk
1,2019,500570,0.00,11.0,36,212420,1,MS,Navirai,NaN,NaN,1,Masculino,212420.0,Analista de suporte


# Gerando ID para Sigla_UF

In [7]:
# Create id for sigla_uf
df["sigla_uf_id"] = df["sigla_uf"].astype("category").cat.codes
df.head(2)


,ano,id_uf_municipio,remuneracao_media,desligamento_id,idade,ocupacao_id,sexo_id,sigla_uf,sigla_municipio,valor_remuneracao_media,raca_cor_id,chave,valor,cbo_2002,sinonimo,sigla_uf_id
0,2019,500240,997.99,12.0,17,317210,1,MS,Caarapo,NaN,NaN,1,Masculino,317210.0,Analista de service desk,11
1,2019,500570,0.00,11.0,36,212420,1,MS,Navirai,NaN,NaN,1,Masculino,212420.0,Analista de suporte,11


## Gerando Regiões por Sigla

In [8]:
# SEPARATE sigla_uf by regions

# https://pt.wikipedia.org/wiki/Regi%C3%B5es_do_Brasil
# https://pt.wikipedia.org/wiki/Unidades_federativas_do_Brasil
REGIOES = {
    "Norte": ["AC", "AP", "AM", "PA", "RO", "RR", "TO"],
    "Nordeste": ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"],
    "Centro-Oeste": ["GO", "MT", "MS"],
    "Sudeste": ["ES", "MG", "RJ", "SP"],
    "Sul": ["PR", "RS", "SC"]
}


In [9]:
df_regiao = df.copy()

# add col Regioes to df_ana with REGIOES
for regiao, siglas in REGIOES.items():
    df_regiao.loc[df["sigla_uf"].isin(
        siglas), "regiao"] = regiao

df_regiao["regiao_id"] = df_regiao["regiao"].astype("category").cat.codes
df = df_regiao
df.head(2)


,ano,id_uf_municipio,remuneracao_media,desligamento_id,idade,ocupacao_id,sexo_id,sigla_uf,sigla_municipio,valor_remuneracao_media,raca_cor_id,chave,valor,cbo_2002,sinonimo,sigla_uf_id,regiao,regiao_id
0,2019,500240,997.99,12.0,17,317210,1,MS,Caarapo,NaN,NaN,1,Masculino,317210.0,Analista de service desk,11,Centro-Oeste,0
1,2019,500570,0.00,11.0,36,212420,1,MS,Navirai,NaN,NaN,1,Masculino,212420.0,Analista de suporte,11,Centro-Oeste,0


# Desligamento

In [10]:
# Create new col desligamento from desligament_id, if != 0, so "nao", otherwise "sim"
df["desligamento"] = df["desligamento_id"].apply(
    lambda x: 0 if x != 0 else 1)
df.head(2)



,ano,id_uf_municipio,remuneracao_media,desligamento_id,idade,ocupacao_id,sexo_id,sigla_uf,sigla_municipio,valor_remuneracao_media,raca_cor_id,chave,valor,cbo_2002,sinonimo,sigla_uf_id,regiao,regiao_id,desligamento
0,2019,500240,997.99,12.0,17,317210,1,MS,Caarapo,NaN,NaN,1,Masculino,317210.0,Analista de service desk,11,Centro-Oeste,0,0
1,2019,500570,0.00,11.0,36,212420,1,MS,Navirai,NaN,NaN,1,Masculino,212420.0,Analista de suporte,11,Centro-Oeste,0,0


# Selecionando colunas necessárias

In [11]:
df = df[["ano", "sigla_uf_id", "regiao", "regiao_id", "sigla_uf", "remuneracao_media", 'desligamento', "idade",
         "ocupacao_id", "sinonimo", "sexo_id", "valor"]]

df = df.rename(columns={
    "sinonimo": "ocupacao",
    "valor": "sexo"
    })

df.head(2)

,ano,sigla_uf_id,regiao,regiao_id,sigla_uf,remuneracao_media,desligamento,idade,ocupacao_id,ocupacao,sexo_id,sexo
0,2019,11,Centro-Oeste,0,MS,997.99,0,17,317210,Analista de service desk,1,Masculino
1,2019,11,Centro-Oeste,0,MS,0.00,0,36,212420,Analista de suporte,1,Masculino


# Salvando arquivo

In [12]:
df.to_parquet("./output/gold/rais_TODOS_ANOS_comJoin_RAIS_VINC_PUB.parquet.gzip", index=False, compression="gzip")